## Import All modules

In [ ]:
# importing sys
import sys

# adding Folder_2 to the system path
sys.path.insert(0, '/home/jeongeun/test_env/Open-Set-Object-Detection')

In [ ]:
# AI2 THOR
!pkill thor_*
import ai2thor
from ai2thor.controller import Controller,BFSController
from ai2thor.platform import CloudRendering
from ithor_tools.vis_tool import *
from ithor_tools.transform import cornerpoint_projection,depth2world
from ithor_tools.map2 import single_scenemap
from ithor_tools.landmark_utils import gather,vis_panorama,Word_Dict,choose_ladmark
import random
import math
# from IPython.display import display
# from moviepy.editor import ImageSequenceClip,VideoFileClip

# Co occurance module
from co_occurance.comet_co import co_occurance_score
from co_occurance.move import co_occurance_based_schedular

# Planning Module
from RRT import gridmaprrt as rrt
from RRT import gridmaprrt_pathsmoothing as smoothing
# from IPython.display import Image as IM

In [ ]:
# Detector module
from data.phase_1 import load_voc_instances,VOC_CLASS_NAMES
import torch
import cv2
import copy
import matplotlib.pyplot as plt
from structures.box import Boxes
from engine.predictor import DefaultPredictor

from  config.config import get_cfg
from model.rcnn import GeneralizedRCNN
from detector.postprocess import postprocess,plot_openset,plot_candidate

## Matching Module
from detector.query_matching import matcher

## Setup AI2THOR

In [ ]:
gridSize=0.05
scene_name = "FloorPlan_Train8_1"
controller = Controller(
    platform = CloudRendering,
    agentMode="locobot",
    visibilityDistance=5.0,
    scene = scene_name,
    gridSize=gridSize,
    movementGaussianSigma=0,
    rotateStepDegrees=90,
    rotateGaussianSigma=0,
    renderClassImage = True,
    renderDepthImage=False,
    renderInstanceSegmentation=False,
    width=300,
    height=300,
    fieldOfView=60
)

In [ ]:
controller.reset(
    # makes the images a bit higher quality
    width=800,
    height=800,

    # Renders several new image modalities
    renderDepthImage=True,
    renderClassImage = True,
    renderSemanticSegmentation=False,
    renderNormalsImage=False
)
scene_bounds = controller.last_event.metadata['sceneBounds']['center']
controller.step(
    action="AddThirdPartyCamera",
    position=dict(x=scene_bounds['x'], y=5.0, z=scene_bounds['z']),
    rotation=dict(x=90, y=0, z=0),
    orthographic=True,
    orthographicSize= 5.0, fieldOfView=100,
    skyboxColor="white"
)
controller.step(dict(action='GetReachablePositions'))
rstate = controller.last_event.metadata['actionReturn']

controller.step(
    action="Teleport",
    position = rstate[100]
)

pos = controller.last_event.metadata['agent']['position']
pos = [pos['x'],pos['z']]
objects = controller.last_event.metadata['objects']

In [ ]:
df = show_objects_table(objects)
df

## Make Object Query

In [ ]:
landmarks,visible_landmark_name = choose_ladmark(objects)
landmark_cat = [Word_Dict[l] for l in visible_landmark_name]
# query_object = random.choice(objects)
query_object = objects[15] #20
query_object_name = query_object['objectType']

print(query_object_name)

In [ ]:
landmarks

## Setup Co occurance

In [ ]:
co_occurance_scoring = co_occurance_score('cuda:1')
co_occurance_scoring.landmark_init(landmark_cat)


In [ ]:
scene_bounds = controller.last_event.metadata['sceneBounds']['cornerPoints']
scene_bounds = cornerpoint_projection(scene_bounds)

sm = single_scenemap(scene_bounds,rstate,stepsize = 0.1,
                landmark_names=visible_landmark_name,landmarks=landmarks)
sm.plot_landmarks(controller,show=False)
landmark_config = dict(name=visible_landmark_name,color = sm.landmark_colors)
imshow_grid = sm.plot(controller.last_event.metadata['agent']['position'],query_object['position'])
plot_frames(controller.last_event,imshow_grid,landmark_config)

## Setup Detector

In [ ]:
'''
config file
'''

print(torch.cuda.device_count())
torch.cuda.set_device(0)
print(torch.cuda.current_device())
cfg = get_cfg()
cfg.merge_from_file('../Open-Set-Object-Detection/config_files/voc.yaml')
cfg.MODEL.SAVE_IDX=19 #22
cfg.MODEL.RPN.USE_MDN=False
cfg.log = False 
cfg.MODEL.ROI_HEADS.USE_MLN = True
cfg.MODEL.ROI_HEADS.AUTO_LABEL = False
cfg.MODEL.ROI_HEADS.AF = 'baseline'
cfg.MODEL.RPN.AUTO_LABEL = False
cfg.MODEL.ROI_BOX_HEAD.USE_FD = False
cfg.MODEL.RPN.AUTO_LABEL_TYPE = 'sum'
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 21
cfg.INPUT.RANDOM_FLIP = "none"
cfg.MODEL.ROI_HEADS.UNCT = True
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.2
cfg.PATH = '../Open-Set-Object-Detection'

# cfg.merge_from_list(args.opts)
RPN_NAME = 'mdn' if cfg.MODEL.RPN.USE_MDN else 'base'
ROI_NAME = 'mln' if cfg.MODEL.ROI_HEADS.USE_MLN else 'base'
MODEL_NAME = RPN_NAME + ROI_NAME
# cfg.merge_from_list(args.opts)
cfg.freeze()
# wandb.init(config=cfg,tags= 'temp',name = 'temp',project='temp')

In [ ]:
device = 'cuda:1'
model = GeneralizedRCNN(cfg,device = device).to(device)
state_dict = torch.load('../Open-Set-Object-Detection/ckpt/{}/{}_{}_15000.pt'.format(cfg.MODEL.ROI_HEADS.AF,cfg.MODEL.SAVE_IDX,MODEL_NAME),map_location=device)
pretrained_dict = {k: v for k, v in state_dict.items() if k in model.state_dict()}
model.load_state_dict(pretrained_dict)

predictor = DefaultPredictor(cfg,model)

In [ ]:
'''
Example of Openset Detection
'''
VOC_CLASS_NAMES = (
    "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat",
    "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person",
    "pottedplant", "sheep", "sofa", "train", "tvmonitor", 'unknown'
)

img = controller.last_event.cv2img
pred = predictor(img)
pred_boxes, pred_classes,_ = postprocess(pred)
plot_openset(img,pred_boxes,pred_classes,VOC_CLASS_NAMES)

In [ ]:
'''
Load Matcher
'''
query_matcher = matcher(query_object_name,threshold=29,device = device)

## Co-occurance Score

In [ ]:
thres = 0.6
res = co_occurance_scoring.score(query_object_name)
print(res,visible_landmark_name)
if max(res)<thres:
    thres = max(res)-0.2

In [ ]:
'''
Move to inital position
'''
controller.step(
    action="Teleport",
    position = rstate[100]
)

In [ ]:
schedular = co_occurance_based_schedular(landmarks,visible_landmark_name,num_loi=2)
schedular.get_node(sm,controller,res,thres)
controller.step(
    action="Teleport",
    position = rstate[100]
)
schedular.get_edge(controller)
path = schedular.optimize()
vis_visit_landmark(query_object,path,controller,sm,landmark_config)

In [ ]:
from eval_ithor.reset import get_min_dis

min_dis = get_min_dis(query_object,controller,sm,schedular)
print(min_dis)

## Move To landmark

In [ ]:
rrtplanner = rrt.RRT(controller = controller, expand_dis=0.1,max_iter=10000,goal_sample_rate=20)
d2w = depth2world()


In [ ]:
'''
Move to inital position
'''
controller.step(
    action="Teleport",
    position = rstate[100]
)

In [ ]:
def detect(frames,single_pos,gt_boxes,controller,predictor,matcher,show):
    patch = np.zeros((0,256,256,3),dtype=np.uint8)
    map_p = []
    sucesses = 0
    for frame,pos,gt_box in zip(frames,single_pos,gt_boxes):
        pred = predictor(frame)
        pred_boxes, pred_classes, unk_only = postprocess(pred)
        if show:
            plot_openset(frame,pred_boxes,pred_classes,VOC_CLASS_NAMES)
        show_patch,candidate_boxes,sucess = matcher.matching_score(frame,pred_boxes[unk_only],gt_box)
        sucesses += torch.sum(sucess).item()
        if len(show_patch):
            DEPTH = controller.last_event.depth_frame
            COLOR = controller.last_event.frame.astype(np.uint8)
            map_points = d2w.object_coord(candidate_boxes,DEPTH,COLOR
                                ,pos['pos'],pos['rot'])
            patch = np.concatenate((patch,show_patch),axis=0)
            map_p += map_points
    return patch,map_p,sucesses

In [ ]:
total_patch = np.zeros((0,256,256,3),dtype=np.uint8)
total_mappoints = []
angle = 60
step = 3
total_success = 0
total_path_len = 0
for p in path[1:]:
    print(p)
    pos = controller.last_event.metadata['agent']['position']
    rrtplanner.set_start(pos)
    rrtplanner.set_goal(p[0])
    print("start planning")
    local_path = rrtplanner.planning(animation=False)
    try:
        smoothpath = smoothing.path_smoothing(rrtplanner,40,verbose=False)
    except:
        smoothpath = local_path
    print("end planning")
    rrtplanner.plot_path(smoothpath)
    
    flag,path_len,frames = rrtplanner.go_with_teleport(smoothpath,maxspeed=0.2)
    total_path_len += path_len
    # video = ImageSequenceClip(frames, fps=10)
    # video.write_gif('temp.gif')
    # with open('temp.gif','rb') as file:
    #     display(IM(file.read(),width = 300))
    
    pos = controller.last_event.metadata['agent']['position']
    controller.step(
        action="Teleport",
        position = pos, rotation = dict(x=0,y=p[1]-angle/2,z=0)
            )
    print("end move")
    imshow_grid = sm.plot(pos,query_object['position'])
    plot_frames(controller.last_event,imshow_grid,landmark_config)
    frames, single_pos,gt_boxes,gt_vis = gather(controller,[query_object['objectId']],step=step,angle=angle)
    print('gt_vis?',gt_vis)
    candidate_patches, candidate_map_points,sucesses = detect(frames,single_pos,gt_boxes,controller,predictor,query_matcher,show=gt_vis>0)
    total_patch = np.concatenate((total_patch,candidate_patches),axis=0)
    total_mappoints += candidate_map_points
    total_success += sucesses
    vis_panorama(frames,res=angle)
    print(len(total_patch))
    if len(total_patch)>30 or gt_vis:
        break
print("Sucess?",total_success>0)
print("Total Path Length", total_path_len)
plot_candidate(total_patch,total_mappoints,query_object_name,sm)

In [ ]:
SPL = (total_success>0)*min_dis/total_path_len
print(SPL)

In [ ]:
total_success